In [1]:
import pyspark.sql.functions as func
from time import time

In [2]:
spark

In [3]:
spark.sql("show databases").show()

+-------------------+
|       databaseName|
+-------------------+
|            default|
|        default_new|
|       defaultgsdfg|
|               demo|
|         infofusion|
|infofusion_mapr_orc|
|             sample|
|             thcp10|
|             tpch10|
+-------------------+



In [4]:
spark.conf.set("spark.driver.maxResultSize", 0)


In [5]:
sqlString = "select o_orderpriority from sample.orders";
ds = spark.sql(sqlString)

In [6]:
ds.show(4)

+---------------+
|o_orderpriority|
+---------------+
|          5-LOW|
|       1-URGENT|
|          5-LOW|
|          5-LOW|
+---------------+
only showing top 4 rows



In [7]:
# crosstab pivoted specifications
# dimension->1 (from the same table) "o_orderstatus"
# measure-> 1 count from orders
# pivot -> same table "o_orderpriority"
# dimension from orders(TPCH) as bdaordersperf 
t1 = time()


print('**********Printing Pivot Column Discrete values***************')
sqlString = "select o_orderpriority from sample.orders";
ds = spark.sql(sqlString)

distinctds = ds.distinct().sort("o_orderpriority")
distinctds=distinctds.limit(1000)
for value in distinctds.collect():
   print value[0]

dforders = spark.table("sample.orders")
for column in dforders.columns:
   dforders = dforders.withColumnRenamed(column,"perf_orders_" + column)
rows = dforders.groupBy("perf_orders_o_orderstatus").pivot("perf_orders_o_orderpriority").agg(func.count('perf_orders_ROW_NUM')).sort("perf_orders_o_orderstatus")

rows1 = dforders.groupBy("perf_orders_o_orderstatus").agg(func.count('perf_orders_ROW_NUM')).sort("perf_orders_o_orderstatus") 
#rows1.show();

rows = rows.join(rows1,rows.perf_orders_o_orderstatus == rows1.perf_orders_o_orderstatus,"inner")

rows = rows.limit(1000);
print('****** Printing Actual Dataset ******************************')
for row in rows.collect():
    print row[0], row[1],  row[2], row[3], row[4], row[5], row[6], row[7]

print('*************Total count ************************************')
print rows.count()

totals = dforders.groupBy().pivot("perf_orders_o_orderpriority").agg(func.count('perf_orders_ROW_NUM'))
firstRowTotals = totals.first()
print('****** Printing Total  Dataset ******************************')
print firstRowTotals[0],firstRowTotals[1],firstRowTotals[2],firstRowTotals[3],firstRowTotals[4]

totals1 = dforders.groupBy().agg(func.count('perf_orders_ROW_NUM'))
firstRow1 = totals1.first()
print firstRow1[0]

print('****** Printing Time taken ******************************')
print time() - t1

**********Printing Pivot Column Discrete values***************
1-URGENT
2-HIGH
3-MEDIUM
4-NOT SPECIFIED
5-LOW
****** Printing Actual Dataset ******************************
F 1462766 1461800 1462276 1461756 1460586 F 7309184
O 1464013 1461240 1459703 1461850 1460658 O 7307464
P 76314 77021 76961 76654 76402 P 383352
*************Total count ************************************
3
****** Printing Total  Dataset ******************************
3003093 3000061 2998940 3000260 2997646
15000000
****** Printing Time taken ******************************
30.2608239651


In [18]:
# crosstab pivoted specifications
# dimension->1 (from the same table) "o_orderstatus"
# measure-> 1 mean('o_totalprice') from orders
# pivot -> same table "o_orderpriority"
# dimension from orders(TPCH) as bdaordersperf 

t1 = time()

print('**********Printing Pivot Column Discrete values***************')
sqlString = "select o_orderpriority from sample.bdaordersperf";
ds = spark.sql(sqlString)
distinctds = ds.distinct().sort("o_orderpriority")
distinctds=distinctds.limit(1000)
for value in distinctds.collect():
   print value[0]


dforders = spark.table("sample.bdaordersperf")
for column in dforders.columns:
   dforders = dforders.withColumnRenamed(column,"perf_orders_" + column)
rows = dforders.groupBy("perf_orders_o_orderstatus")\
             .pivot("perf_orders_o_orderpriority").agg(func.mean('perf_orders_o_totalprice'))\
             .sort("perf_orders_o_orderstatus")

#rows.show();
rows1 = dforders.groupBy("perf_orders_o_orderstatus")\
             .agg(func.mean('perf_orders_o_totalprice'))\
             .sort("perf_orders_o_orderstatus") 
#rows1.show();
rows = rows.join(rows1,rows.perf_orders_o_orderstatus == rows1.perf_orders_o_orderstatus,"inner")
#rows.show();
rows = rows.limit(1000);
print('****** Printing Actual Dataset ******************************')
for row in rows.collect():
    print row[0], row[1],  row[2], row[3], row[4], row[5], row[6], row[7]
    
print('*************Total count ************************************')
print rows.count()

totals = dforders.groupBy().pivot("perf_orders_o_orderpriority").agg(func.mean('perf_orders_o_totalprice'))

print('****** Printing Total  Dataset ******************************')
firstRowTotals = totals.first()
print firstRowTotals[0],firstRowTotals[1],firstRowTotals[2],firstRowTotals[3],firstRowTotals[4]

totals1 = dforders.groupBy().agg(func.mean('perf_orders_o_totalprice'))
firstRow1 = totals1.first()
print firstRow1[0]

****** Printing Time taken ******************************
print time() - t1

**********Printing Pivot Column Discrete values***************
1-URGENT
2-HIGH
3-MEDIUM
4-NOT SPECIFIED
5-LOW
****** Printing Actual Dataset ******************************
F 150217.953676 150360.2419 150107.554346 150142.456041 150265.569962 F 150218.740521
O 150218.541297 150173.386023 150191.375195 150240.910075 150093.931562 O 150183.652375
P 184776.741132 185048.370787 184401.316909 184914.398212 185078.166727 P 184843.545656
*************Total count ************************************
3
****** Printing Total  Dataset ******************************
151096.441152 151159.783341 151028.424989 151078.819384 151069.216228
151086.54605
19.4574680328


In [8]:
# crosstab pivoted specifications
# dimension->1 (from the same table) "PropertyType"
# measure->2 count from orders, sum(l_discount)--> Lineitem table
# pivot -> same table "o_orderpriority"
# dimension from o_orderstatus
t1 = time()

print('**********Printing Pivot Column Discrete values***************')
sqlString = "select o_orderpriority from sample.orders";
ds = spark.sql(sqlString)

distinctds = ds.distinct().sort("o_orderpriority")
distinctds=distinctds.limit(1000)
for value in distinctds.collect():
   print value[0]
#################################
#   ACTUAL and TOTAL RESULTS (1)
################################

dforders = spark.table("sample.orders")
for column in dforders.columns:
   dforders = dforders.withColumnRenamed(column,"perf_orders_" + column)
rows = dforders.groupBy("perf_orders_o_orderstatus")\
             .pivot("perf_orders_o_orderpriority").agg(func.count('perf_orders_ROW_NUM'))\
             .sort("perf_orders_o_orderstatus")


rows1 = dforders.groupBy("perf_orders_o_orderstatus")\
             .agg(func.count('perf_orders_ROW_NUM'))\
             .sort("perf_orders_o_orderstatus") 

rows = rows.join(rows1,'perf_orders_o_orderstatus',"inner")
#rows.show();

######################
# TOTAL (1)
###############################
totals = dforders.groupBy().pivot("perf_orders_o_orderpriority").agg(func.count('perf_orders_ROW_NUM'))
firstRowTotals = totals.first()
print('****** Printing Total  Dataset ******************************')
print firstRowTotals[0],firstRowTotals[1],firstRowTotals[2],firstRowTotals[3],firstRowTotals[4]

totals1 = dforders.groupBy().agg(func.count('perf_orders_ROW_NUM'))
firstRow1 = totals1.first()
print firstRow1[0]


#################################
#   ACTUAL and TOTAL RESULTS (2)
################################
dflineitem = spark.table("sample.lineitem")
for column in dflineitem.columns:
    dflineitem = dflineitem.withColumnRenamed(column, "perf_lineitem_" + column);
#dflineitem.printSchema();
dfJoin1 = dforders.join(dflineitem,dforders.perf_orders_o_orderkey == dflineitem.perf_lineitem_l_orderkey,"left")
dfJoin1 = dfJoin1.dropDuplicates(['perf_orders_o_orderstatus','perf_orders_o_orderpriority','perf_lineitem_ROW_NUM'])
rows21 = dfJoin1.groupBy("perf_orders_o_orderstatus")\
             .pivot("perf_orders_o_orderpriority").agg(func.sum('perf_lineitem_l_discount'))\
             .sort("perf_orders_o_orderstatus")

dfJoin2 = dfJoin1.dropDuplicates(['perf_orders_o_orderstatus','perf_lineitem_ROW_NUM'])
rows22 = dfJoin2.groupBy("perf_orders_o_orderstatus")\
             .agg(func.sum('perf_lineitem_l_discount'))\
             .sort("perf_orders_o_orderstatus") 
        
actualdsrows2 = rows21.join(rows22,rows21.perf_orders_o_orderstatus == rows22.perf_orders_o_orderstatus,"inner")
#actualdsrows2.show()

######################
# TOTAL (2)
###############################
totals = dfJoin1.groupBy().pivot("perf_orders_o_orderpriority").agg(func.sum('perf_lineitem_l_discount'))
firstRowTotals = totals.first()
print('****** Printing Total  Dataset for second measure ******************************')
print firstRowTotals[0],firstRowTotals[1],firstRowTotals[2],firstRowTotals[3],firstRowTotals[4]

totals1 = dfJoin2.groupBy().agg(func.sum('perf_lineitem_l_discount'))
firstRow1 = totals1.first()
print firstRow1[0]


###########################################
#   COMBINING ACTUAL DS of TWO MESAURES
#############################################
rows.cache()
actualdsrows2.cache()
actualds = rows.join(actualdsrows2,'perf_orders_o_orderstatus',"inner")

actualds = actualds.limit(1000);
print('****** Printing Actual Dataset ******************************')
for row in actualds.collect():
    print row[0], row[1],  row[2], row[3], row[4], row[5], row[6], row[7],row[8], row[9],  row[10], row[11], row[12]

    
print('*************Total count ************************************')
print rows.count()

print time() - t1

**********Printing Pivot Column Discrete values***************
1-URGENT
2-HIGH
3-MEDIUM
4-NOT SPECIFIED
5-LOW
****** Printing Total  Dataset ******************************
3003093 3000061 2998940 3000260 2997646
15000000
****** Printing Total  Dataset for second measure ******************************
600577.46 600095.81 599572.4 599881.83 599245.74
2999373.24
****** Printing Actual Dataset ******************************
F 1462766 1461800 1462276 1461756 1460586 7309184 290778.13 290766.35 290567.15 290456.62 290399.31 F
O 1464013 1461240 1459703 1461850 1460658 7307464 291158.15 290450.17 290219.85 290624.73 290167.16 O
P 76314 77021 76961 76654 76402 383352 18641.18 18879.29 18785.4 18800.48 18679.27 P
*************Total count ************************************
3
558.369824886


In [8]:
# crosstab pivoted specifications [3 measures]
# dimension->1 (from the same table) "PropertyType"
# measure->2 count from orders, sum(l_discount), mean(o_totalprice)-->order, Lineitem table
# pivot -> same table "o_orderpriority"
# dimension from o_orderstatus
t1 = time()

print('**********Printing Pivot Column Discrete values***************')
sqlString = "select o_orderpriority from sample.orders";
ds = spark.sql(sqlString)

distinctds = ds.distinct().sort("o_orderpriority")
distinctds=distinctds.limit(1000)
for value in distinctds.collect():
   print value[0]
#################################
#   ACTUAL and TOTAL RESULTS (1)
################################

dforders = spark.table("sample.orders")
for column in dforders.columns:
   dforders = dforders.withColumnRenamed(column,"perf_orders_" + column)
rows = dforders.groupBy("perf_orders_o_orderstatus")\
             .pivot("perf_orders_o_orderpriority").agg(func.count('perf_orders_ROW_NUM'))\
             .sort("perf_orders_o_orderstatus")


rows1 = dforders.groupBy("perf_orders_o_orderstatus")\
             .agg(func.count('perf_orders_ROW_NUM'))\
             .sort("perf_orders_o_orderstatus") 

rows = rows.join(rows1,'perf_orders_o_orderstatus',"inner")
#rows.show();

######################
# TOTAL (1)
###############################
totals = dforders.groupBy().pivot("perf_orders_o_orderpriority").agg(func.count('perf_orders_ROW_NUM'))
firstRowTotals = totals.first()
print('****** Printing Total  Dataset ******************************')
print firstRowTotals[0],firstRowTotals[1],firstRowTotals[2],firstRowTotals[3],firstRowTotals[4]

totals1 = dforders.groupBy().agg(func.count('perf_orders_ROW_NUM'))
firstRow1 = totals1.first()
print firstRow1[0]


#################################
#   ACTUAL and TOTAL RESULTS (2)
################################
dflineitem = spark.table("sample.lineitem")
for column in dflineitem.columns:
    dflineitem = dflineitem.withColumnRenamed(column, "perf_lineitem_" + column);
#dflineitem.printSchema();
dfJoin1 = dforders.join(dflineitem,dforders.perf_orders_o_orderkey == dflineitem.perf_lineitem_l_orderkey,"left")
dfJoin1 = dfJoin1.dropDuplicates(['perf_orders_o_orderstatus','perf_orders_o_orderpriority','perf_lineitem_ROW_NUM'])
rows21 = dfJoin1.groupBy("perf_orders_o_orderstatus")\
             .pivot("perf_orders_o_orderpriority").agg(func.sum('perf_lineitem_l_discount'))\
             .sort("perf_orders_o_orderstatus")

dfJoin2 = dfJoin1.dropDuplicates(['perf_orders_o_orderstatus','perf_lineitem_ROW_NUM'])
rows22 = dfJoin2.groupBy("perf_orders_o_orderstatus")\
             .agg(func.sum('perf_lineitem_l_discount'))\
             .sort("perf_orders_o_orderstatus") 
        
actualdsrows2 = rows21.join(rows22,rows21.perf_orders_o_orderstatus == rows22.perf_orders_o_orderstatus,"inner")
#actualdsrows2.show()

######################
# TOTAL (2)
###############################
totals = dfJoin1.groupBy().pivot("perf_orders_o_orderpriority").agg(func.sum('perf_lineitem_l_discount'))
firstRowTotals = totals.first()
print('****** Printing Total  Dataset for second measure ******************************')
print firstRowTotals[0],firstRowTotals[1],firstRowTotals[2],firstRowTotals[3],firstRowTotals[4]

totals1 = dfJoin2.groupBy().agg(func.sum('perf_lineitem_l_discount'))
firstRow1 = totals1.first()
print firstRow1[0]

#################################
#   ACTUAL and TOTAL RESULTS (3)
################################

dforders = spark.table("sample.orders")
for column in dforders.columns:
   dforders = dforders.withColumnRenamed(column,"perf_orders_" + column)
rowsm3 = dforders.groupBy("perf_orders_o_orderstatus")\
             .pivot("perf_orders_o_orderpriority").agg(func.mean('perf_orders_o_totalprice'))\
             .sort("perf_orders_o_orderstatus")


rows1m3 = dforders.groupBy("perf_orders_o_orderstatus")\
             .agg(func.mean('perf_orders_o_totalprice'))\
             .sort("perf_orders_o_orderstatus") 

rowsm3 = rowsm3.join(rows1m3,'perf_orders_o_orderstatus',"inner")
#rows.show();

######################
# TOTAL (3)
###############################
totals = dforders.groupBy().pivot("perf_orders_o_orderpriority").agg(func.mean('perf_orders_o_totalprice'))
firstRowTotals = totals.first()
print('****** Printing Total  Dataset for measure 3 ******************************')
print firstRowTotals[0],firstRowTotals[1],firstRowTotals[2],firstRowTotals[3],firstRowTotals[4]

totals1 = dforders.groupBy().agg(func.mean('perf_orders_o_totalprice'))
firstRow1 = totals1.first()
print firstRow1[0]


###########################################
#   COMBINING ACTUAL DS of TWO MESAURES
#############################################
rows.cache()
actualdsrows2.cache()
actualds = rows.join(actualdsrows2,'perf_orders_o_orderstatus',"inner")

actualds.cache()
rowsm3.cache()
actualds = actualds.join(rowsm3,'perf_orders_o_orderstatus',"inner")

actualds = actualds.limit(1000);
print('****** Printing Actual Dataset ******************************')
for row in actualds.collect():
    print row[0], row[1],  row[2], row[3], row[4], row[5], row[6], row[7],row[8], row[9],  row[10], row[11], row[12], row[13], row[14],  row[15], row[16], row[17], row[18], row[19]

    
print('*************Total count ************************************')
print rows.count()

print time() - t1

**********Printing Pivot Column Discrete values***************
1-URGENT
2-HIGH
3-MEDIUM
4-NOT SPECIFIED
5-LOW
****** Printing Total  Dataset ******************************
3003093 3000061 2998940 3000260 2997646
15000000
****** Printing Total  Dataset for second measure ******************************
600577.46 600095.81 599572.4 599881.83 599245.74
2999373.24
****** Printing Total  Dataset for measure 3 ******************************
151096.441152 151159.783341 151028.424989 151078.819384 151069.216228
151086.54605
****** Printing Actual Dataset ******************************
F 1462766 1461800 1462276 1461756 1460586 7309184 290778.13 290766.35 290567.15 290456.62 290399.31 F 1452967.56 150217.953676 150360.2419 150107.554346 150142.456041 150265.569962 150218.740521
O 1464013 1461240 1459703 1461850 1460658 7307464 291158.15 290450.17 290219.85 290624.73 290167.16 O 1452620.06 150218.541297 150173.386023 150191.375195 150240.910075 150093.931562 150183.652375
P 76314 77021 76961 76654

In [9]:
dforders = spark.table("sample.orders")
for column in dforders.columns:
   dforders = dforders.withColumnRenamed(column,"perf_orders_" + column)
rows = dforders.groupBy("perf_orders_o_orderstatus")\
             .pivot("perf_orders_o_orderpriority").agg(func.count('perf_orders_ROW_NUM'))\
             .sort("perf_orders_o_orderstatus")


In [10]:
rows.show()

+-------------------------+--------+-------+--------+---------------+-------+
|perf_orders_o_orderstatus|1-URGENT| 2-HIGH|3-MEDIUM|4-NOT SPECIFIED|  5-LOW|
+-------------------------+--------+-------+--------+---------------+-------+
|                        F| 1462766|1461800| 1462276|        1461756|1460586|
|                        O| 1464013|1461240| 1459703|        1461850|1460658|
|                        P|   76314|  77021|   76961|          76654|  76402|
+-------------------------+--------+-------+--------+---------------+-------+



In [11]:
dflineitem = spark.table("sample.lineitem")
for column in dflineitem.columns:
    dflineitem = dflineitem.withColumnRenamed(column, "perf_lineitem_" + column);
#dflineitem.printSchema();
dfJoin1 = dforders.join(dflineitem,dforders.perf_orders_o_orderkey == dflineitem.perf_lineitem_l_orderkey,"left")
dfJoin1 = dfJoin1.dropDuplicates(['perf_orders_o_orderstatus','perf_orders_o_orderpriority','perf_lineitem_ROW_NUM'])
rows21 = dfJoin1.groupBy("perf_orders_o_orderstatus")\
             .pivot("perf_orders_o_orderpriority").agg(func.sum('perf_lineitem_l_discount'))\
             .sort("perf_orders_o_orderstatus")



In [12]:
rows21.show()

+-------------------------+-----------------+-----------------+------------------+------------------+------------------+
|perf_orders_o_orderstatus|         1-URGENT|           2-HIGH|          3-MEDIUM|   4-NOT SPECIFIED|             5-LOW|
+-------------------------+-----------------+-----------------+------------------+------------------+------------------+
|                        F|290778.1299999097|290766.3499999097|   290567.14999991| 290456.6199999101|290399.30999990983|
|                        O|291158.1499999092|  290450.16999991|290219.84999991016| 290624.7299999096| 290167.1599999102|
|                        P|18641.18000000002|18879.29000000002|18785.400000000005|18800.480000000014|18679.270000000015|
+-------------------------+-----------------+-----------------+------------------+------------------+------------------+



In [13]:
totals = dforders.groupBy().pivot("perf_orders_o_orderpriority").agg(func.count('perf_orders_ROW_NUM'))

In [14]:
totals.show()

+--------+-------+--------+---------------+-------+
|1-URGENT| 2-HIGH|3-MEDIUM|4-NOT SPECIFIED|  5-LOW|
+--------+-------+--------+---------------+-------+
| 3003093|3000061| 2998940|        3000260|2997646|
+--------+-------+--------+---------------+-------+



In [15]:
firstRow1 = totals1.first()
print firstRow1[0]

2999373.24
